In [ ]:
import sys
import csv
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np
import pickle 
import pprint
import requests
from datetime import date, datetime
from random import *
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import os
import datetime
import shutil
import glob

In [ ]:
pp = pprint.PrettyPrinter(indent=4)

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('window-size=1000x500')
options.add_argument("start-maximized")
options.add_argument("disable-infobars")
# options.add_argument("--disable-extensions")
# options.add_argument("disable-gpu")



In [ ]:
# 데이터 불러오기
print(os.getcwd())
camping = pd.read_csv('../file/camping.csv')

In [ ]:
camping.head(3)

In [180]:
def post_count(num, df, name, tag_nm):
    rows_list = []
    tag_list = df[tag_nm].values.tolist()
    
    today = datetime.datetime.today() 
    today_ymd = today.strftime('%Y%m%d')

    path = '../file/' 
    new_path = ''.join([path, today_ymd])
    file_nm = '_'.join(['camping', today_ymd])
    
    output_nm = ''.join([path, "_", file_nm, ".csv"])
    mergeoutput_nm = ''.join([new_path, "_",file_nm, ".csv"])
    
    cols = ['no','camp_nm','hashtag','num_posts','date_time']
    df_out = pd.DataFrame(columns=cols)
    
    try:
        os.mkdir(new_path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)
    
    chrome_path = '/usr/local/bin/chromedriver'
    driver = webdriver.Chrome(chrome_path, options=options)

    for i in range(num,len(tag_list)):
        driver.get('https://www.instagram.com/explore/tags/'+tag_list[i]+'/?hl=en') 
        time.sleep(3) 
        source = driver.page_source 
        data = bs(source, 'html.parser') 
        tag = data.find("span",{"class": "g47SY "}) 
        try: 
            val = tag.text 
            val = int(val.replace(',' , '')) 
        except AttributeError: 
            val = 0

        info = [i, df[name][i], tag_list[i], val, today]
        dict1 = dict(zip(cols,info))
#         dict1 = {"no":i,
#                  "camp_nm":df[name][i],
#                  "hashtag":tag_list[i],
#                  "num_of_post":val,
#                  "date_time":today}
        df_out.append(dict1, ignore_index=True)
        print(dict1)
        time.sleep(1)
        
        if i%50 == 49:
            driver.quit()
            df_out.to_csv(path+file_nm+f"{i:0>5}"+'.csv', index=False, header=False)
            
            df_out = pd.DataFrame(columns=cols)
            
            time.sleep(7)
            
            driver = webdriver.Chrome(chrome_path, options=options)

    df_out.to_csv(path+file_nm+f"{i:0>5}"+'.csv')
    driver.quit()
    
    with open(output_nm, 'wb') as outfile:
        for filename in glob.glob(path+file_nm+'*.csv'):
    #         print(filename)
            if filename == output_nm:
                # don't want to copy the output into the output
                continue

            with open(filename, 'rb') as readfile:
                readfile.readline()
                shutil.copyfileobj(readfile, outfile)

    outfile = pd.read_csv(output_nm, names=cols, 
                          header=None, index_col='id')
    outfile.to_csv(mergeoutput_nm, index=False)

In [ ]:
post_count(0, camping, 'name','name_modify')

Creation of the directory ../file/ failed
{'no': 0, 'camp_nm': '한탄강소나무캠핑', 'hashtag': '한탄강소나무캠핑', 'num_posts': 9, 'date_time': datetime.datetime(2020, 6, 6, 11, 24, 38, 942519)}
{'no': 1, 'camp_nm': '한탄강관광지오토캠핑장', 'hashtag': '한탄강관광지오토캠핑장', 'num_posts': 2667, 'date_time': datetime.datetime(2020, 6, 6, 11, 24, 38, 942519)}


In [176]:
cols = ['no','camp_nm','hashtag','num_posts','date_time']
pd.DataFrame(columns=cols)

,no,camp_nm,hashtag,num_posts,date_time
